In [0]:
#get the file name from the adf
fileName = dbutils.widgets.get('fileName')
#fileName = 'Product.csv'
fileNameWithoutExt = fileName.split('.')[0]
print(fileNameWithoutExt)

Product


In [0]:
import pyspark.sql.functions as F
#from datetime import datetme as dt

#Just change all the values here based on the resource name you have created in your environemnt and workspace.

sqlDbName = 'morgonsqldatabase'
dbUserName = 'Rahul'
passwordKey = 'sqldbpasswordkey'
stgAccountSASTokenKey = 'sasforstorage2'
landingFileName =fileName #'Product'  #dbutils.widgets.get('Product')
databricksScopeName ='morgondatabricksscope'
dbServer = 'morgonserver'
dbServerPortNumber ='1433'
storageContainer ='inputdata'
storageAccount='morgonstorage'
landingMountPoint ='/mnt'


In [0]:
if not any(mount.mountPoint == landingMountPoint for mount in dbutils.fs.mounts()):
    dbutils.fs.mount( source = 'wasbs://{}@{}.blob.core.windows.net'.format(storageContainer, storageAccount), mount_point= landingMountPoint, extra_configs ={'fs.azure.sas.{}.{}.blob.core.windows.net'.format(storageContainer,storageAccount):dbutils.secrets.get(scope = databricksScopeName, key= stgAccountSASTokenKey)})
    print('Mounted the storage account successfully')
else:
    print('Storage account already mounted')

Storage account already mounted


In [0]:
#connect to Azure SQL DB
dbPassword = dbutils.secrets.get(scope = databricksScopeName, key= passwordKey)
serverurl = 'jdbc:sqlserver://{}.database.windows.net:{};database={};user={};'.format(dbServer, dbServerPortNumber,sqlDbName,dbUserName)
connectionProperties = {
    'password':dbPassword,
    'driver':'com.microsoft.sqlserver.jdbc.SQLServerDriver'
}
df = spark.read.jdbc(url = serverurl, table = 'dbo.FileDetailsFormat', properties= connectionProperties)
display(df)


FileNo,FileName,ColumnName,ColumnDateFormat,ColumnIsNull,ModifiedDate
1,Product,StartDate,MM-dd-yyyy,true,2012-06-18T22:34:09Z
1,Product,EndDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
1,Product,CreateDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
1,Product,ModifiedDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
2,ProductDescription,ModifiedDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
2,ProductDescription,StartDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
2,ProductDescription,EndDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
3,CustomerDetail,CreateDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
3,CustomerDetail,ActiveDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z


In [0]:


errorFlag = False

if df.filter(df["FileName"] == fileNameWithoutExt).count() > 0:
# check for the duplicate values in df
    df1 = spark.read.csv('/mnt/landing/'+fileName, inferSchema=True, header=True)
    df2 = df.filter(df.FileName==fileNameWithoutExt).select('ColumnName','ColumnDateFormat' )

    #totalcount in csv file
    totalcount = df1.count()

    #distinct count in csv file
    distinctCount = df1.distinct().count()

    #formatcount from csv file to check if all rows are converted properly
    rows = df2.collect()
    for r in rows:
        colName = r[0]
        colFormat =r[1]
    formatCount =df1.filter(F.to_date(colName, colFormat).isNotNull() ==True).count()

    if distinctCount !=totalcount:
        errorMessage = 'Duplication Found.'
        errorFlag = True
    elif formatCount == totalcount:
        errorFlag = True
        errorMessage = ' DateFormate is incorrect for {} '.format(colName)
    elif (distinctCount !=totalcount) and (formatCount == totalcount):
        errorFlag = True
        errorMessage = 'Duplication found and DateFormate is incorrect for {} '.format(colName)
    else:
        errorMessage = 'Moving file to staging'


    
else:
    errorFlag = True
    errorMessage = 'Moving file to rejected because file Name is incorrect'



if errorFlag:
    dbutils.fs.mv('/mnt/landing/'+fileName,'/mnt/rejected/'+fileName )
    dbutils.notebook.exit('{"errorFlag": "true", "errorMessage":"'+errorMessage +'"}')
else:
    dbutils.fs.mv('/mnt/landing/'+fileName,'/mnt/staging/'+fileName )
    dbutils.notebook.exit('{"errorFlag": "false", "errorMessage":"'+errorMessage +'"}')